In [ ]:
import pandas as pd
import numpy as np
import re
import yfinance as yf
from datetime import datetime, timedelta

In [ ]:
board_data = pd.read_csv('/content/hynix_board_merged_sorted_2024-01-01_to_2025-05-26(in).csv')
board_data.head()

,date,title,link,author,view,like,dislike,body
0,2025-05-26 9:24,앤비디아 발표 저가,https://finance.naver.com/item/board_read.nave...,cjh0****,116,0,0,H20 만들어서 중국에 판다 호재
1,2025-05-26 9:23,ai대장주 sk하이닉스 곧 삼십만원 갑니...[1],https://finance.naver.com/item/board_read.nave...,mjh8****,158,3,2,삼성전자도 못따라오는 ai기술 sk하이닉스에 몰빵하세요\n\n하반기 실적 선반영합니...
2,2025-05-26 9:22,코스피지수 봉우리가 부담,https://finance.naver.com/item/board_read.nave...,sam7****,44,0,0,하이닉스는 좋은기업인데\n코스피지수 꼭대기에 20만원밑이라\n부담스럽다\n지수꺽이면...
3,2025-05-26 9:20,"김부선 ""현관에서 바지 벗고 뛰어들던 사...",https://finance.naver.com/item/board_read.nave...,myeo****,23,0,0,술그만쳐먹고 병원가봐라
4,2025-05-26 9:19,★★ 곧 상장폐지 공시 예정,https://finance.naver.com/item/board_read.nave...,ienv****,97,0,5,이딴것도 주식이라 ㅋㅋㅋㅋㅋ


In [ ]:
# 2. date 컬럼 타입 확인
print(f"원본 date 타입: {board_data['date'].dtype}")
print("원본 date 샘플:")
print(board_data['date'].head())

원본 date 타입: object
원본 date 샘플:
0    2025-05-26 9:24
1    2025-05-26 9:23
2    2025-05-26 9:22
3    2025-05-26 9:20
4    2025-05-26 9:19
Name: date, dtype: object


In [ ]:
# 3. 문자열이라면 정리 후 변환
if board_data['date'].dtype == 'object':
    board_data['date'] = board_data['date'].str.replace('\n', ' ')
    board_data['date'] = board_data['date'].str.replace('  ', ' ')

In [ ]:
# 4. datetime 변환
board_data['date'] = pd.to_datetime(board_data['date'], errors='coerce')

In [ ]:
# 5. 정렬
board_data = board_data.sort_values('date').reset_index(drop=True)

In [ ]:

print("최종 날짜 범위:")
print(f"최소: {board_data['date'].min()}")
print(f"최대: {board_data['date'].max()}")

최종 날짜 범위:
최소: 2024-01-01 05:15:00
최대: 2025-05-26 09:24:00


### 데이터 전처리 1-1. 텍스트 전처리

In [ ]:
def clean_text(text):
    if pd.isna(text):
        return ""
    text = re.sub(r'<[^>]+>', '', text)  # HTML 태그 제거
    text = re.sub(r'http[s]?://\S+', '', text)  # URL 제거
    text = re.sub(r'[^\w\s가-힣]', ' ', text)  # 특수문자 제거
    text = ' '.join(text.split())  # 공백 정리
    return text

In [ ]:
board_data['cleaned_title'] = board_data['title'].apply(clean_text)
board_data['cleaned_body'] = board_data['body'].apply(clean_text)

In [ ]:
board_data.head()

,date,title,link,author,view,like,dislike,body,cleaned_title,cleaned_body
0,2024-01-01 05:15:00,Happy New Year 청룡의 갑진년...,https://finance.naver.com/item/board_read.nave...,isya****,2899,0,1,♡1개사 상장 주식에 대해 대주주 양도세 완화 10억에서 50억으로 상향 시행 출발...,Happy New Year 청룡의 갑진년,1개사 상장 주식에 대해 대주주 양도세 완화 10억에서 50억으로 상향 시행 출발 ...
1,2024-01-01 09:11:00,군바리,https://finance.naver.com/item/board_read.nave...,yunk****,2429,0,0,여서 매달 1주씩 밖에 못사는 게 한이다..\n이번달 월급 들어오면 1주 더 사야지,군바리,여서 매달 1주씩 밖에 못사는 게 한이다 이번달 월급 들어오면 1주 더 사야지
2,2024-01-01 09:19:00,세입자 임대료에 부가가치세 전가 행위는 ...[1],https://finance.naver.com/item/board_read.nave...,gogo****,2521,3,0,불경기에 먹고 살기도 급급한\n세입자들\n임대료에 부가 가치세를 전가하는\n임대주들...,세입자 임대료에 부가가치세 전가 행위는 1,불경기에 먹고 살기도 급급한 세입자들 임대료에 부가 가치세를 전가하는 임대주들에게 ...
3,2024-01-01 10:07:00,한반도를 난장판 만든 양키와 부역자들!![2],https://finance.naver.com/item/board_read.nave...,jky4****,2457,3,8,"청약통장도 모른 윤완용에 경제는 뻔한것,\n현명한 외교로 국익을 최대화 시키기는 커...",한반도를 난장판 만든 양키와 부역자들 2,청약통장도 모른 윤완용에 경제는 뻔한것 현명한 외교로 국익을 최대화 시키기는 커녕 ...
4,2024-01-01 10:10:00,내손목아지 자르고 싶다[3],https://finance.naver.com/item/board_read.nave...,lmt8****,2773,9,2,78.000원에 전량 매도한게\n한스럽다\n\n반이라도 남길걸 6천주도\n넘는걸 홧...,내손목아지 자르고 싶다 3,78 000원에 전량 매도한게 한스럽다 반이라도 남길걸 6천주도 넘는걸 홧김에 다 ...


In [ ]:
stock_data = pd.read_csv('/content/skhynix_2024_2025.csv')

In [ ]:
stock_data.head(10)

,Date,Open,High,Low,Close,Volume,Change,MA3,MA5
0,2024-01-02,139700,142800,139700,142400,2147458,0.006360,NaN,NaN
1,2024-01-03,140000,140800,136800,136800,3257820,-0.039326,NaN,NaN
2,2024-01-04,136800,138800,135800,136400,2661970,-0.002924,138533.333333,NaN
3,2024-01-05,135800,137500,135800,137500,1846781,0.008065,136900.000000,NaN
4,2024-01-08,137500,137900,135400,136000,2498302,-0.010909,136633.333333,137820.0
5,2024-01-09,139000,139500,136600,137400,3473806,0.010294,136966.666667,136820.0
6,2024-01-10,137700,138100,132100,133500,3769252,-0.028384,135633.333333,136160.0
7,2024-01-11,132400,137100,132400,136000,3594909,0.018727,135633.333333,136080.0
8,2024-01-12,136400,137300,133400,134100,1878915,-0.013971,134533.333333,135400.0
9,2024-01-15,134800,135100,133300,134100,1858867,0.000000,134733.333333,135020.0


In [ ]:
# 게시글 날짜 전처리
board_data['date'] = pd.to_datetime(board_data['date'])
stock_data['Date'] = pd.to_datetime(stock_data['Date'])
stock_data['Change'] = stock_data['Change'] * 100

In [ ]:
stock_data.head()

,Date,Open,High,Low,Close,Volume,Change,MA3,MA5
0,2024-01-02,139700,142800,139700,142400,2147458,0.636042,NaN,NaN
1,2024-01-03,140000,140800,136800,136800,3257820,-3.932584,NaN,NaN
2,2024-01-04,136800,138800,135800,136400,2661970,-0.292398,138533.333333,NaN
3,2024-01-05,135800,137500,135800,137500,1846781,0.806452,136900.000000,NaN
4,2024-01-08,137500,137900,135400,136000,2498302,-1.090909,136633.333333,137820.0


# 주식데이터 라벨 생성

In [ ]:
# 다음 거래일 수익률로 라벨 생성
def create_labels(board_row, stock_df):
    post_date = board_row['date'].date()

    # 다음 거래일 찾기
    future_prices = stock_df[stock_df['Date'].dt.date > post_date]
    if len(future_prices) == 0:
        return np.nan

    next_return = future_prices.iloc[0]['Change']

    # 라벨링: 0=하락, 1=보합, 2=상승
    if next_return < -0.01:  # 1% 이상 하락
        return 0
    elif next_return > 0.01:   # 1% 이상 상승
        return 2
    else:                      # 보합
        return 1


In [ ]:
# 라벨 생성
board_data['label'] = board_data.apply(lambda x: create_labels(x, stock_data), axis=1)

In [ ]:
# 라벨 분포 확인
label_counts = board_data['label'].value_counts().sort_index()
print(f"라벨 분포:")
print(f"하락(0): {label_counts[0]:,}개")
print(f"보합(1): {label_counts[1]:,}개")
print(f"상승(2): {label_counts[2]:,}개")

라벨 분포:
하락(0): 56,377개
보합(1): 2,107개
상승(2): 58,303개


In [ ]:
missing_labels = board_data['label'].isna().sum()
total_data = len(board_data)

In [ ]:
print(f"전체 데이터: {total_data:,}개")
print(f"라벨 있는 데이터: {board_data['label'].notna().sum():,}개")
print(f"라벨 없는 데이터: {missing_labels:,}개")
print(f"라벨 없는 비율: {missing_labels/total_data*100:.1f}%")

전체 데이터: 116,849개
라벨 있는 데이터: 116,787개
라벨 없는 데이터: 62개
라벨 없는 비율: 0.1%


In [ ]:
# 라벨 없는 데이터 제거
board_data = board_data.dropna(subset=['label'])
board_data['label'] = board_data['label'].astype(int)

<ipython-input-21-c8cf9b4b9490>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  board_data['label'] = board_data['label'].astype(int)


In [ ]:
# 의미없는 텍스트 필터링
def is_meaningful_text(title, body):
    if pd.isna(title) or pd.isna(body):
        return False

    # 너무 짧은 텍스트
    if len(title.strip()) < 2 or len(body.strip()) < 5:
        return False

    # 이모티콘/반복문자만 있는 텍스트
    import re
    title_clean = re.sub(r'[^\w가-힣]', '', title)
    if len(title_clean) < 2:
        return False

    # 반복문자 (ㅋㅋㅋ, ㅠㅠㅠ 등)
    if re.match(r'^(.)\1{2,}$', title_clean):
        return False

    return True


In [ ]:
# 필터링 적용
quality_mask = board_data.apply(lambda x: is_meaningful_text(x['cleaned_title'], x['cleaned_body']), axis=1)
filtered_data = board_data[quality_mask].copy()

In [ ]:
print(f"필터링 전: {len(board_data):,}개")
print(f"필터링 후: {len(filtered_data):,}개")
print(f"제거된 데이터: {len(board_data) - len(filtered_data):,}개 ({(1-len(filtered_data)/len(board_data))*100:.1f}%)")


필터링 전: 116,787개
필터링 후: 103,225개
제거된 데이터: 13,562개 (11.6%)


In [ ]:
# 라벨 분포 재확인
print(f"\n필터링 후 라벨 분포:")
print(filtered_data['label'].value_counts().sort_index())


필터링 후 라벨 분포:
label
0    49867
1     1878
2    51480
Name: count, dtype: int64


In [ ]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 35.3 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Okt
import re

# 형태소 분석기 초기화
okt = Okt()

# 불용어 정의
stopwords = {'의', '가', '이', '은', '는', '을', '를', '에', '와', '과', '도', '로', '으로',
             '에서', '까지', '부터', '께서', '에게', '한테', '하고', '그리고', '그래서',
             '하지만', '그런데', '또한', '및', '등', '것', '수', '때', '곳', '점'}

In [ ]:
# 토큰화 함수
def tokenize_text(text):
    if pd.isna(text) or not text.strip():
        return []

    # 형태소 분석 (명사, 형용사, 동사만)
    tokens = okt.pos(text, stem=True)
    meaningful_tokens = []

    for word, pos in tokens:
        if pos in ['Noun', 'Adjective', 'Verb'] and len(word) > 1:
            if word not in stopwords:
                meaningful_tokens.append(word)

    return meaningful_tokens


In [ ]:
filtered_data['title_tokens'] = filtered_data['cleaned_title'].apply(tokenize_text)
filtered_data['body_tokens'] = filtered_data['cleaned_body'].apply(tokenize_text)

In [ ]:
filtered_data.head()

,date,title,link,author,view,like,dislike,body,cleaned_title,cleaned_body,label,title_tokens,body_tokens
0,2024-01-01 05:15:00,Happy New Year 청룡의 갑진년...,https://finance.naver.com/item/board_read.nave...,isya****,2899,0,1,♡1개사 상장 주식에 대해 대주주 양도세 완화 10억에서 50억으로 상향 시행 출발...,Happy New Year 청룡의 갑진년,1개사 상장 주식에 대해 대주주 양도세 완화 10억에서 50억으로 상향 시행 출발 ...,2,"[청룡, 갑진년]","[개사, 상장, 주식, 대해, 주주, 도세, 완화, 상향, 시행, 출발, 공매도, ..."
1,2024-01-01 09:11:00,군바리,https://finance.naver.com/item/board_read.nave...,yunk****,2429,0,0,여서 매달 1주씩 밖에 못사는 게 한이다..\n이번달 월급 들어오면 1주 더 사야지,군바리,여서 매달 1주씩 밖에 못사는 게 한이다 이번달 월급 들어오면 1주 더 사야지,2,[바리],"[여서, 매달, 살다, 하다, 이번, 월급, 들어오다, 사다]"
2,2024-01-01 09:19:00,세입자 임대료에 부가가치세 전가 행위는 ...[1],https://finance.naver.com/item/board_read.nave...,gogo****,2521,3,0,불경기에 먹고 살기도 급급한\n세입자들\n임대료에 부가 가치세를 전가하는\n임대주들...,세입자 임대료에 부가가치세 전가 행위는 1,불경기에 먹고 살기도 급급한 세입자들 임대료에 부가 가치세를 전가하는 임대주들에게 ...,2,"[입자, 임대료, 부가가치세, 전가, 행위]","[불경기, 먹다, 살기, 입자, 임대료, 부가, 가다, 전가, 하다, 임대, 중벌,..."
3,2024-01-01 10:07:00,한반도를 난장판 만든 양키와 부역자들!![2],https://finance.naver.com/item/board_read.nave...,jky4****,2457,3,8,"청약통장도 모른 윤완용에 경제는 뻔한것,\n현명한 외교로 국익을 최대화 시키기는 커...",한반도를 난장판 만든 양키와 부역자들 2,청약통장도 모른 윤완용에 경제는 뻔한것 현명한 외교로 국익을 최대화 시키기는 커녕 ...,2,"[한반도, 난장판, 만들다, 양키, 부역]","[청약, 통장, 모르다, 윤완용, 경제, 뻔하다, 현명하다, 외교, 국익, 최대, ..."
4,2024-01-01 10:10:00,내손목아지 자르고 싶다[3],https://finance.naver.com/item/board_read.nave...,lmt8****,2773,9,2,78.000원에 전량 매도한게\n한스럽다\n\n반이라도 남길걸 6천주도\n넘는걸 홧...,내손목아지 자르고 싶다 3,78 000원에 전량 매도한게 한스럽다 반이라도 남길걸 6천주도 넘는걸 홧김에 다 ...,2,"[손목, 아지, 자르다, 싶다]","[전량, 매도, 하다, 스럽다, 남다, 넘다, 홧김, 던지다, 가슴, 쓰다, 에코,..."


In [ ]:
import urllib.request

In [ ]:
base_url = "https://raw.githubusercontent.com/park1200656/knu_senti_dict/master/"
urllib.request.urlretrieve(base_url + "pos_pol_word.txt", "pos_pol_word.txt")
urllib.request.urlretrieve(base_url + "neg_pol_word.txt", "neg_pol_word.txt")

('neg_pol_word.txt', <http.client.HTTPMessage at 0x7f8a4df6d0d0>)

In [ ]:
# 감성사전 로드
with open('pos_pol_word.txt', 'r', encoding='utf-8') as f:
    pos_words = set(word.strip() for word in f.readlines())

with open('neg_pol_word.txt', 'r', encoding='utf-8') as f:
    neg_words = set(word.strip() for word in f.readlines())


In [ ]:
# 주식 특화 감성 단어 추가
stock_positive = {'상승', '떡상', '급등', '폭등', '반등', '매수', '호재', '대박',
                  '좋다', '추천', '믿음', '희망', 'HBM', '수주', '실적', '성장'}

stock_negative = {'하락', '떡락', '급락', '폭락', '매도', '손절', '악재', '최악',
                  '걱정', '위험', '망함', '침체', '손실', '적자', '공급과잉'}

In [ ]:
pos_words.update(stock_positive)
neg_words.update(stock_negative)

print(f"긍정 단어: {len(pos_words)}개")
print(f"부정 단어: {len(neg_words)}개")

긍정 단어: 4893개
부정 단어: 9854개


In [ ]:
# 감성 점수 계산 함수
def calculate_sentiment_score(tokens):
    if not tokens:
        return 0

    pos_count = sum(1 for token in tokens if token in pos_words)
    neg_count = sum(1 for token in tokens if token in neg_words)

    return (pos_count - neg_count) / len(tokens)


In [ ]:
# 감성 점수 계산
filtered_data['title_sentiment'] = filtered_data['title_tokens'].apply(calculate_sentiment_score)
filtered_data['body_sentiment'] = filtered_data['body_tokens'].apply(calculate_sentiment_score)

print("감성 점수 계산 완료")
print(f"제목 감성 평균: {filtered_data['title_sentiment'].mean():.4f}")
print(f"본문 감성 평균: {filtered_data['body_sentiment'].mean():.4f}")

감성 점수 계산 완료
제목 감성 평균: -0.0116
본문 감성 평균: -0.0157


In [ ]:
# 분산 (얼마나 다양한 감성 점수를 가지는가)
print(f"제목 감성 표준편차: {filtered_data['title_sentiment'].std():.4f}")

# 0이 아닌 값들만 (실제 감성 있는 글들)
non_zero = filtered_data[filtered_data['title_sentiment'] != 0]['title_sentiment']
print(f"감성 있는 글 비율: {len(non_zero)/len(filtered_data)*100:.1f}%")
print(f"감성 있는 글들의 평균 절댓값: {non_zero.abs().mean():.4f}")

제목 감성 표준편차: 0.2219
감성 있는 글 비율: 27.3%
감성 있는 글들의 평균 절댓값: 0.3665


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from datetime import datetime

In [ ]:
# TF-IDF 벡터화
def tokens_to_text(tokens):
    return ' '.join(tokens) if tokens else ''

# 토큰을 다시 텍스트로 변환
filtered_data['title_text'] = filtered_data['title_tokens'].apply(tokens_to_text)
filtered_data['body_text'] = filtered_data['body_tokens'].apply(tokens_to_text)

# TF-IDF 벡터화 (상위 1000개 단어만)
tfidf = TfidfVectorizer(max_features=1000, min_df=5, max_df=0.7)
title_tfidf = tfidf.fit_transform(filtered_data['title_text'])

print(f"TF-IDF 특성 수: {title_tfidf.shape[1]}개")

# 메타 특성 생성
filtered_data['title_length'] = filtered_data['cleaned_title'].str.len()
filtered_data['body_length'] = filtered_data['cleaned_body'].str.len()
filtered_data['title_token_count'] = filtered_data['title_tokens'].apply(len)
filtered_data['body_token_count'] = filtered_data['body_tokens'].apply(len)

# 시간 특성
filtered_data['hour'] = filtered_data['date'].dt.hour
filtered_data['day_of_week'] = filtered_data['date'].dt.dayofweek
filtered_data['is_weekend'] = filtered_data['day_of_week'].isin([5, 6]).astype(int)

# 주식 키워드 빈도
stock_keywords = ['매수', '매도', '상승', '하락', '투자', '수익', '손실', '호재', '악재']
for keyword in stock_keywords:
    filtered_data[f'{keyword}_count'] = filtered_data['title_tokens'].apply(lambda x: x.count(keyword))

print("수치적 특성 생성 완료")
print(f"전체 특성 수: {len(filtered_data.columns)}개")

TF-IDF 특성 수: 1000개
수치적 특성 생성 완료
전체 특성 수: 33개


In [ ]:
# 인덱스 리셋 후 다시 진행
filtered_data = filtered_data.reset_index(drop=True)

# TF-IDF도 다시 생성
tfidf_df = pd.DataFrame(title_tfidf.toarray(),
                       columns=[f'tfidf_{i}' for i in range(title_tfidf.shape[1])])

# 특성 데이터 준비
feature_cols = ['title_sentiment', 'body_sentiment', 'title_length', 'body_length',
                'title_token_count', 'body_token_count', 'hour', 'day_of_week', 'is_weekend']

stock_keyword_cols = [col for col in filtered_data.columns if col.endswith('_count')]
feature_cols.extend(stock_keyword_cols)

X_meta = filtered_data[feature_cols]
X_combined = pd.concat([X_meta.reset_index(drop=True), tfidf_df], axis=1)
y = filtered_data['label']

# 시간순 분할
split_date = filtered_data['date'].quantile(0.8)
train_mask = filtered_data['date'] <= split_date

X_train = X_combined[train_mask]
X_test = X_combined[~train_mask]
y_train = y[train_mask]
y_test = y[~train_mask]

print(f"훈련 데이터: {len(X_train)}개")
print(f"테스트 데이터: {len(X_test)}개")

# 모델 학습
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print(f"정확도: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))

훈련 데이터: 82580개
테스트 데이터: 20645개
정확도: 0.5349
              precision    recall  f1-score   support

           0       0.62      0.49      0.54     11255
           1       0.00      0.00      0.00       595
           2       0.47      0.63      0.54      8795

    accuracy                           0.53     20645
   macro avg       0.36      0.37      0.36     20645
weighted avg       0.54      0.53      0.53     20645



# 문제 상황: 보합의 케이스가 너무 적어 학습을 못하여 정확도가 0에 수렴.

In [ ]:
# class_weight='balanced' 사용
rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print(f"정확도: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))

정확도: 0.5356
              precision    recall  f1-score   support

           0       0.61      0.50      0.55     11255
           1       0.00      0.00      0.00       595
           2       0.47      0.62      0.54      8795

    accuracy                           0.54     20645
   macro avg       0.36      0.37      0.36     20645
weighted avg       0.54      0.54      0.53     20645



# KoBERT 모델링으로 일단 넘어가기..

In [ ]:
!pip install transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 844.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from torch.utils.data import Dataset
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import torch.nn as nn

In [ ]:
!pip install transformers torch

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from torch.utils.data import Dataset
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import torch.nn as nn

# KoBERT 토크나이저와 모델 로드
model_name = "skt/kobert-base-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# 데이터셋 클래스
class StockDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# 클래스 가중치 계산
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)
print(f"클래스 가중치: {class_weights}")

# 수정된 가중치 적용 트레이너
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        loss_fct = nn.CrossEntropyLoss(weight=class_weights_tensor.to(model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

# 데이터 준비
train_texts = filtered_data[train_mask]['cleaned_title'].tolist()
test_texts = filtered_data[~train_mask]['cleaned_title'].tolist()

train_dataset = StockDataset(train_texts, y_train.tolist(), tokenizer)
test_dataset = StockDataset(test_texts, y_test.tolist(), tokenizer)

# 훈련 설정
training_args = TrainingArguments(
    output_dir='./kobert_weighted',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=500,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
)

# 트레이너 생성 및 훈련
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


클래스 가중치: [ 0.71290445 21.45492336  0.64487915]


Epoch,Training Loss,Validation Loss
1,1.184700,0.802349
2,1.148400,0.807718
3,1.132600,0.807096


TrainOutput(global_step=15486, training_loss=1.15368528586556, metrics={'train_runtime': 5750.4442, 'train_samples_per_second': 43.082, 'train_steps_per_second': 2.693, 'total_flos': 1.629592952698368e+16, 'train_loss': 1.15368528586556, 'epoch': 3.0})

In [ ]:
# KoBERT 토크나이저와 모델 로드
model_name = "skt/kobert-base-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)


In [ ]:

# 데이터셋 클래스
class StockDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# 데이터 준비 (제목만 사용)
train_texts = filtered_data[train_mask]['cleaned_title'].tolist()
test_texts = filtered_data[~train_mask]['cleaned_title'].tolist()

train_dataset = StockDataset(train_texts, y_train.tolist(), tokenizer)
test_dataset = StockDataset(test_texts, y_test.tolist(), tokenizer)

print(f"훈련 데이터: {len(train_dataset)}개")
print(f"테스트 데이터: {len(test_dataset)}개")

# 훈련 설정
training_args = TrainingArguments(
    output_dir='./kobert_stock',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=500,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
)

# 트레이너 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# 훈련 시작
trainer.train()

In [ ]:
# KoBERT 모델 예측 및 평가
from sklearn.metrics import classification_report, accuracy_score

# 예측 수행
trainer.model.eval()
predictions = trainer.predict(test_dataset)
y_pred_kobert = predictions.predictions.argmax(-1)

# 결과 출력
print(f"KoBERT 정확도: {accuracy_score(y_test, y_pred_kobert):.4f}")
print("\nKoBERT 분류 리포트:")
print(classification_report(y_test, y_pred_kobert))

# 기존 Random Forest와 비교
print(f"\n성능 비교:")
print(f"Random Forest: {accuracy_score(y_test, y_pred):.4f}")
print(f"KoBERT: {accuracy_score(y_test, y_pred_kobert):.4f}")

KoBERT 정확도: 0.5452

KoBERT 분류 리포트:
              precision    recall  f1-score   support

           0       0.55      1.00      0.71     11255
           1       0.00      0.00      0.00       595
           2       0.00      0.00      0.00      8795

    accuracy                           0.55     20645
   macro avg       0.18      0.33      0.24     20645
weighted avg       0.30      0.55      0.38     20645


성능 비교:
Random Forest: 0.5356
KoBERT: 0.5452


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


정확도는 54% (Random Forest와 비슷)

하지만 클래스 1, 2를 전혀 예측 못함 (precision/recall 0.00)

모든 예측을 클래스 0(하락)으로만 함

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import torch.nn as nn

# 1. 클래스 가중치 계산
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

print(f"클래스 가중치: {class_weights}")

# 2. 커스텀 트레이너 클래스 생성
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # 가중치 적용된 손실 함수
        loss_fct = nn.CrossEntropyLoss(weight=class_weights_tensor.to(model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

# 3. 가중치 적용된 트레이너로 다시 훈련
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# 4. 다시 훈련
trainer.train()